In [33]:
import torch
from pde import LHS_pde, RHS_pde, true_solution, generate_boundary_points, cal_l2_relative_error

In [32]:
# mlp
class MLP(torch.nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.linear1 = torch.nn.Linear(dim_in, dim_hidden)
        self.linear2 = torch.nn.Linear(dim_hidden, dim_hidden)
        self.linear3 = torch.nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = torch.tanh(self.linear1(x))
        x = torch.tanh(self.linear2(x))
        x = self.linear3(x)
        return x

In [36]:
# pinn train
def pinn_train(model, optimizer, dim_set, epoch, num_bd_pts, num_interior_pts, num_test_pts, right, left):
    model.train()
    optimizer.zero_grad()
    # interior points
    x_interior = torch.rand(num_interior_pts, dim_set) * (right - left) + left
    x_interior.requires_grad = True
    y_interior = model(x_interior)
    LHS_interior = LHS_pde(y_interior, x_interior, dim_set)
    RHS_interior = RHS_pde(x_interior)
    loss_interior = torch.mean((LHS_interior - RHS_interior)**2)
    # boundary points
    x_boundary = generate_boundary_points(num_bd_pts, dim_set, right, left)
    x_boundary.requires_grad = True
    y_boundary = model(x_boundary)
    LHS_boundary = LHS_pde(y_boundary, x_boundary, dim_set)
    RHS_boundary = RHS_pde(x_boundary)
    loss_boundary = torch.mean((LHS_boundary - RHS_boundary)**2)

    # loss
    loss = loss_interior + 100 * loss_boundary
    loss.backward()
    optimizer.step()
    # print
    if epoch % 100 == 0:
        # cal l2 error
        x_test = torch.rand(num_test_pts, dim_set) * (right - left) + left
        y_test = model(x_test)
        true_test = true_solution(x_test)
        l2_error = cal_l2_relative_error(y_test, true_test)
        print('epoch: %d, loss: %.4e, l2_error: %.4e' % (epoch, loss.item(), l2_error.item()))

In [37]:
#train
dim_set = 2
dim_hidden = 20
dim_out = 1
num_epoch = 10000
num_bd_pts = 100
num_interior_pts = 10000
num_test_pts = 10000
right = 1
left = 0
model = MLP(dim_set, dim_hidden, dim_out)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(num_epoch):
    pinn_train(model, optimizer, dim_set, epoch, num_bd_pts, num_interior_pts, num_test_pts, right, left)

epoch: 0, loss: 4.2209e+02, l2_error: 5.7386e-01
epoch: 100, loss: 1.3926e+01, l2_error: 4.8070e+00
epoch: 200, loss: 4.2048e+00, l2_error: 4.9405e+00
epoch: 300, loss: 2.5853e+00, l2_error: 5.0906e+00
epoch: 400, loss: 1.9050e+00, l2_error: 5.3034e+00
epoch: 500, loss: 1.3585e+00, l2_error: 5.5064e+00
epoch: 600, loss: 7.8554e-01, l2_error: 5.7232e+00
epoch: 700, loss: 7.5676e-01, l2_error: 5.8709e+00
epoch: 800, loss: 6.5538e-01, l2_error: 5.9516e+00


KeyboardInterrupt: 